In [14]:
from dotenv import load_dotenv
load_dotenv(override=True)

import os
import json

from langchain.vectorstores import FAISS
# FAISS (Facebook AI Similarity Search) to biblioteka opracowana przez Facebook AI Research do wydajnego wyszukiwania podobieństw i grupowania gęstych wektorów.

from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [7]:
VECTOR_STORE_PATH = "memory.index"
MEMORY_PATH = "memory.md"

def get_vector_store():
    # stworzenie memory.index jeśli katalog nie istnieje
    if os.path.exists(VECTOR_STORE_PATH):
        return FAISS.load_local(VECTOR_STORE_PATH, embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
    
    # załadowanie pliku do dokumentu
    loader = TextLoader(MEMORY_PATH)
    memory = loader.load()[0]
    documents = [Document(page_content=content) for content in memory.page_content.split("\n\n")]

    # konwersja tekstu na embeddingi, stworzenie indeksów, stworzenie i zapisanie obiektu FAISS
    store = FAISS.from_documents(documents, OpenAIEmbeddings())
    store.save_local(VECTOR_STORE_PATH)
    return store

`FAISS.from_documents()` pobiera listę dokumentów i:
- generuje embeddingi przy użyciu modelu do embeddingów OpenAI, 
- tworzy indeks wektorów FAISS (domyślnie IndexFLatL2)
- przechowuje embeddingi i dokumenty w indeksie
- zwraca przeszukiwalny obiekt FAISS.

In [12]:
query = "Do you know the name of Adam's dog?"

# wyszukanie powiązanych dokumentów przez similarity search
vector_store = get_vector_store()
context = vector_store.similarity_search_with_score(query, k=1)
context_document, context_score = context[0] # wzięcie top1 dokumentu i jego wyniku


chat = ChatOpenAI()
response = chat.invoke([
    SystemMessage(f"""
        Answer questions as truthfully using the context below and nothing more. If you don't know the answer, say "don't know".
        context###{context_document.page_content if context else ''}###
    """),
    HumanMessage(query),
])

print(response.content)

Yes, the name of Adam's dog is Alexa.


Trzeba trzymać się języka, w którym stworzona jest nasza baza wiedzy. Gdy dokument jest po angielsku, to na pytanie "Czy wiesz jak ma na imie pies Adama?" odpowiedź brzmi "Nie znam imienia psa Adama."

------------------------
Opisywanie bazy dokumentów metadanymi (na podstawie przykładu 11_docs):

In [25]:
# załadowanie pliku do dokumentu
loader = TextLoader(MEMORY_PATH)
memory = loader.load()[0]
documents = [Document(page_content=content) for content in memory.page_content.split("\n\n")]

# przekształcenie listy obiektów 'documents' na listę słowników a dalej na listę takich list z indeksami
documents_dict = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in documents]
docstore = [[str(idx), doc] if idx == 0 else [str(idx), doc] for idx, doc in enumerate(documents_dict)]

docstore_json = json.dumps(docstore)
print(docstore_json)

[["0", {"page_content": "Adam has various skills but describes himself as \"just curious.\"", "metadata": {}}], ["1", {"page_content": "Adam have a dog named Alexa.", "metadata": {}}], ["2", {"page_content": "Adam lives in Krakow with his fianc\u0102\u00a9e and dog.", "metadata": {}}], ["3", {"page_content": "Adam is involved in a couple of projects like eduweb.pl, ahoy.so, easy.tools, overment.com, heyalice.app, automation.house, and more.", "metadata": {}}], ["4", {"page_content": "Adam knows JavaScript and Python very well. He's full-stack engineer.", "metadata": {}}], ["5", {"page_content": "Adam loves music. He listens to Spotify all the time.", "metadata": {}}], ["6", {"page_content": "Adam's nickname is 'overment'.", "metadata": {}}], ["7", {"page_content": "Adam has a youtube channel named 'overment'.", "metadata": {}}], ["8", {"page_content": "Adam is a big fan of Apple products.", "metadata": {}}], ["9", {"page_content": "Adam is a big fan of Tesla cars.", "metadata": {}}]]
